In [1]:
import AlgoCrac as ac

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
from abc import ABC, abstractmethod
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import partial
import matplotlib.pyplot as plt
import datetime
#from indicators import moving_average, exp_moving_average, vwap
#from Signals import *
from AlgoCrac import moving_average, exp_moving_average, vwap
from AlgoCrac.Signals.Signals import *
from AlgoCrac.Signals.utils import *
from AlgoCrac.Strategy.Strategy import *


from tqdm import tqdm

In [3]:



timeframe = "5m"
sym = ['WIX','MMM','GOOGL','TSLA']

data = ac.get_data(sym, start = "2024-06-01",end="2024-06-13",interval = timeframe)

[*********************100%***********************]  4 of 4 completed


In [4]:
data

Adj Close                                      \
                          GOOGL         MMM        TSLA         WIX   
2024-06-03 09:30:00  173.684998  100.180000  181.425003  162.475006   
2024-06-03 09:35:00  173.839996  100.620003  180.722397  160.949997   
2024-06-03 09:40:00  174.195007  100.675003  180.907700  161.000000   
2024-06-03 09:45:00  173.479996  100.320000  181.155106  161.580002   
2024-06-03 09:50:00  173.179993  100.129997  180.955002  161.220001   
...                         ...         ...         ...         ...   
2024-06-12 15:35:00  177.080002  100.816902  177.649994  168.910004   
2024-06-12 15:40:00  177.389999  100.830002  177.500000  168.789993   
2024-06-12 15:45:00  177.520004  101.039001  178.039993  168.880005   
2024-06-12 15:50:00  177.539993  101.345596  177.339996  168.475006   
2024-06-12 15:55:00  177.789993  101.220001  177.190002  168.660004   

                          Close                                      \
                          GOOGL         MMM        TSLA         WIX   
2024-06-03 09:30:00  173.684998  100.180000  181.425003  162.475006   
2024-06-03 09:35:00  173.839996  100.620003  180.722397  160.949997   
2024-06-03 09:40:00  174.195007  100.675003  180.907700  161.000000   
2024-06-03 09:45:00  173.479996  100.320000  181.155106  161.580002   
2024-06-03 09:50:00  173.179993  100.129997  180.955002  161.220001   
...                         ...         ...         ...         ...   
2024-06-12 15:35:00  177.080002  100.816902  177.649994  168.910004   
2024-06-12 15:40:00  177.389999  100.830002  177.500000  168.789993   
2024-06-12 15:45:00  177.520004  101.039001  178.039993  168.880005   
2024-06-12 15:50:00  177.539993  101.345596  177.339996  168.475006   
2024-06-12 15:55:00  177.789993  101.220001  177.190002  168.660004   

                           High              ...         Low              \
                          GOOGL         MMM  ...        TSLA         WIX   
2024-06-03 09:30:00  173.960007  100.510002  ...  177.250000  162.410004   
2024-06-03 09:35:00  173.990005  100.654999  ...  180.199997  160.949997   
2024-06-03 09:40:00  174.520004  100.790001  ...  180.649994  161.000000   
2024-06-03 09:45:00  174.509995  100.669998  ...  180.703201  161.460007   
2024-06-03 09:50:00  173.619995  100.300003  ...  179.876602  160.669998   
...                         ...         ...  ...         ...         ...   
2024-06-12 15:35:00  177.229996  100.959999  ...  177.559998  168.649994   
2024-06-12 15:40:00  177.425003  100.930000  ...  177.250000  168.240005   
2024-06-12 15:45:00  177.679993  101.080002  ...  177.169998  168.630005   
2024-06-12 15:50:00  177.669998  101.379799  ...  177.279999  168.380005   
2024-06-12 15:55:00  178.225006  101.404999  ...  176.960007  168.300003   

                           Open                                       Volume  \
                          GOOGL         MMM        TSLA         WIX    GOOGL   
2024-06-03 09:30:00  172.550003  100.370003  178.350006  162.410004  3791506   
2024-06-03 09:35:00  173.649994  100.220001  181.410004  161.910004   468823   
2024-06-03 09:40:00  173.860001  100.639999  180.740005  161.490005   484045   
2024-06-03 09:45:00  174.179993  100.639999  181.029999  161.544998   292766   
2024-06-03 09:50:00  173.490005  100.279999  181.130005  161.210007   399459   
...                         ...         ...         ...         ...      ...   
2024-06-12 15:35:00  176.899994  100.879997  177.955002  168.830002   248683   
2024-06-12 15:40:00  177.059998  100.820000  177.619995  168.889999   247242   
2024-06-12 15:45:00  177.389999  100.808296  177.490005  168.750000   280198   
2024-06-12 15:50:00  177.529999  101.040001  178.050003  168.800003   507443   
2024-06-12 15:55:00  177.550003  101.360001  177.369995  168.309998  1332300   

                                               
                        MMM     TSLA      WIX  
2024-06-03 09:30:00  161671  4670845   7932.0  


Her is a case where a strategy fails, this is due to intraday days with the signal's periods being larger than the number of timesteps within a single trading day.

In [5]:

period = 120
indicator =  "SMA"
buy_signal = Crossover(period, indicator = indicator, on = "Close", invert = False, eps = 0.2)
sell_signal = Crossover(period, indicator = indicator, on = "Close", invert = True, eps = 0.2)

st = Strategy(buy_signal, sell_signal,
              timeframe=timeframe,
                trade_allocation = 1000,
                  allocation = 10000,
                    negative_allocation= False,
                      side = 'long')

Error: period of the signal is too large for intraday strategy, strategy is not initialized


In [6]:

period = 9
indicator =  "SMA"
v_crossover = Crossover(period, indicator = indicator, on = "Volume", invert = False, eps = 0.2)

period = 12
indicator =  "EMA"
ema_crossover = Crossover(period, indicator = indicator, on = "Close", invert = False)

period = 12
indicator =  "VWAP"
vwap_crossover = Crossover(period, indicator = indicator, on = "Close", invert = False)


below_20 = BasicMetric(condition_value = 20,
                        metric = "last",
                        condition = "lower",
                        on = "Close", # Open, High, Low, Close, Volume
                        invert = False,
                        )



signal_list = [ ema_crossover, vwap_crossover,v_crossover, below_20]
#signal_list = [vwap_crossover]
#signal_list = [v_crossover]

#signal_list = [ ema_crossover]
ma_confl = Confluence(signals_list = signal_list, max_signals = 3, invert = False)

buy_signal = ma_confl
buy_signal.plot_signals(data, signal_type = 'dis')

sell_signal = Confluence(signals_list = signal_list, max_signals = 3, invert = True)

sell_signal.plot_signals(data, signal_type = 'dis')

plot intraday data


plot intraday data


In [7]:
st = Strategy(buy_signal, sell_signal,
              timeframe=timeframe,
                trade_allocation = 1000,
                  allocation = 10000,
                    negative_allocation= False,
                      side = 'long')

## Backtest

In [8]:
st.reset()
bt = Backtest(st, data, fast = False, include_commission= True)
a = bt.run()  
bt.summary() 

  6%|▌         | 37/620 [00:00<00:09, 59.93it/s]c:\Users\liran\OneDrive\Day trading\AlgoCrac\AlgoCrac\Strategy\Strategy.py:242: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`

100%|██████████| 620/620 [00:13<00:00, 46.59it/s]


,allocation,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,win rate
closed,9703.445419,-296.554581,273.0,-1.629421,1.819984,-162.943493,-2.965546,0.999851,7.005495,-0.123894,182,0.456044
all,9703.445419,-296.554581,273.0,-1.629421,1.819984,-162.943493,-2.965546,0.999851,NaN,-3.012513,182,0.456044


* -all- refers to the status at any given time (open and closed trades)
* -closed- refers only to closed trades

bt.status

In [9]:
bt.strategy.closed_trades


,reward,id,symbol,entry time,entry date,entry price,position,max,min,stop loss,commission,exit time,exit date,exit price
0,2.519989,1.0,WIX,11:20:00,2024-06-03,162.779999,6.0,162.779999,162.779999,NaN,1.5,11:25:00,2024-06-03,163.199997
1,2.250000,2.0,WIX,11:35:00,2024-06-03,164.195007,6.0,164.195007,164.195007,NaN,1.5,11:40:00,2024-06-03,164.570007
2,1.800003,3.0,MMM,11:45:00,2024-06-03,99.400002,10.0,99.400002,99.400002,NaN,1.5,11:50:00,2024-06-03,99.580002
3,-4.558563,4.0,TSLA,11:45:00,2024-06-03,179.701706,5.0,179.701706,179.701706,NaN,1.5,11:50:00,2024-06-03,178.789993
4,-0.239960,5.0,WIX,11:45:00,2024-06-03,164.649994,6.0,164.649994,164.649994,NaN,1.5,11:50:00,2024-06-03,164.610001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0.584953,178.0,MMM,15:05:00,2024-06-12,101.105003,9.0,101.105003,101.105003,NaN,1.5,15:10:00,2024-06-12,101.169998
178,-2.599945,180.0,WIX,15:20:00,2024-06-12,170.179993,5.0,170.179993,170.179993,NaN,1.5,15:25:00,2024-06-12,169.660004
179,-3.824997,179.0,GOOGL,15:15:00,2024-06-12,177.634995,5.0,177.634995,176.899994,NaN,1.5,15:30:00,2024-06-12,176.869995
180,0.100021,181.0,GOOGL,15:50:00,2024-06-12,177.529999,5.0,177.529999,177.529999,NaN,1.5,15:55:00,2024-06-12,177.550003


## Fast backtest

In [10]:

st.reset()
bt = Backtest(st, data, fast = True, include_commission= True)
a = bt.run()  
bt.summary() 

,allocation,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,win rate
closed,8639.230789,-303.019592,276.0,-1.646846,1.830974,-165.496437,-3.030196,0.999834,6.983696,-0.123894,184,0.456522
all,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
symbol_summary = bt.strategy.closed_trades[['symbol','reward']].groupby('symbol').sum().copy()
symbol_summary['reward (average)'] = bt.strategy.closed_trades[['symbol','reward']].groupby('symbol').mean()
symbol_summary

,reward,reward (average)
symbol,,
GOOGL,-1.865768,-0.051827
MMM,-6.768143,-0.135363
TSLA,-14.353409,-0.448544
WIX,-4.032272,-0.061095


In [12]:
bt.strategy.closed_trades

,entry time,entry date,exit time,exit date,entry price,exit price,symbol,reward,position,commission,market value,id
0,10:15:00,2024-06-03,10:20:00,2024-06-03,162.990005,162.350006,WIX,-3.839996,6,1.5,974.100037,0
1,11:20:00,2024-06-03,11:25:00,2024-06-03,162.779999,163.199997,WIX,2.519989,6,1.5,979.199982,1
2,11:35:00,2024-06-03,11:40:00,2024-06-03,164.195007,164.570007,WIX,2.250000,6,1.5,987.420044,2
3,11:45:00,2024-06-03,11:50:00,2024-06-03,99.400002,99.580002,MMM,1.800003,10,1.5,995.800018,3
4,11:45:00,2024-06-03,11:50:00,2024-06-03,179.701706,178.789993,TSLA,-4.558563,5,1.5,893.949966,4
...,...,...,...,...,...,...,...,...,...,...,...,...
179,15:05:00,2024-06-12,15:10:00,2024-06-12,101.105003,101.169998,MMM,0.584953,9,1.5,910.529984,179
180,15:15:00,2024-06-12,15:30:00,2024-06-12,177.634995,176.869995,GOOGL,-3.824997,5,1.5,884.349976,180
181,15:20:00,2024-06-12,15:25:00,2024-06-12,170.179993,169.660004,WIX,-2.599945,5,1.5,848.300018,181
182,15:50:00,2024-06-12,15:55:00,2024-06-12,177.529999,177.550003,GOOGL,0.100021,5,1.5,887.750015,182


In [13]:
bt.strategy.open_trades

,reward


In [14]:
pd.concat([bt.strategy.closed_trades.head(),bt.strategy.open_trades.head()], ignore_index=True)

,entry time,entry date,exit time,exit date,entry price,exit price,symbol,reward,position,commission,market value,id
0,10:15:00,2024-06-03,10:20:00,2024-06-03,162.990005,162.350006,WIX,-3.839996,6.0,1.5,974.100037,0.0
1,11:20:00,2024-06-03,11:25:00,2024-06-03,162.779999,163.199997,WIX,2.519989,6.0,1.5,979.199982,1.0
2,11:35:00,2024-06-03,11:40:00,2024-06-03,164.195007,164.570007,WIX,2.250000,6.0,1.5,987.420044,2.0
3,11:45:00,2024-06-03,11:50:00,2024-06-03,99.400002,99.580002,MMM,1.800003,10.0,1.5,995.800018,3.0
4,11:45:00,2024-06-03,11:50:00,2024-06-03,179.701706,178.789993,TSLA,-4.558563,5.0,1.5,893.949966,4.0
